# Access your data in Azure Notebooks

Data is the lifeblood of notebooks. Jupyter itself provides only a runtime environment for a notebook, and thus to do intresting work you need to bring data in from elsewhere.

This notebook provides examples of different ways to import data, all in a format that you can run and experience directly.

- [Use curl to retrieve a file from GitHub](#curl)
- [Use a REST API to retrieve online data](#restapi)
- [Query an Azure SQL database](#azuresql)
- [Access Azure Table Storage](#tablestorage)
- [Access Azure Blobs](#blobs)
  - [Share access to Azure Storage through Shared Access Signatures](#sharedaccess)
- [Other Azure databases (references)](#otherdbs)

# Use `curl` to retrieve a file from the Internet <a name="curl"></a> 

In Python notebooks, you can invoke the command line using `!`, which allows you to download files directly from the Internet using a tool like `curl`, `wget`, and so on. For example, the following `curl` command downloads a file containing oil price data from GitHub, and stores it in the project as the file *oil_price_temp.csv*:

In [ ]:
!curl https://raw.githubusercontent.com/petroleum101/figures/db46e7f48b8aab67a0dfe31696f6071fb7a84f1e/oil_price/oil_price.csv -o ../oil_price_temp.csv

Once the file is in the project, you can load it using any suitable code. For example, you can load it into a pandas dataframe, after which you can work with it however you like:

In [ ]:
import pandas
dataframe_file = pandas.read_csv('../oil_price_temp.csv')
dataframe_file.head()

## Use REST APIs to retrieve online data <a name="restapi"></a>

Generally speaking, the vast amount of data available from the Internet is accessed not through files, but through REST APIs. Fortunately, because a notebook cell can contain whatever code you like, you can use code to send requests and receive JSON data. You can then convert that JSON into whatever format you want to use, such as a pandas dataframe.

The following example is taken from https://dev.socrata.com/foundry/data.cityofnewyork.us/gkne-dk5s, which provides a dataset for 2014 New York Taxis. The sodapy library is used in the code.

In [ ]:
!pip install sodapy

In [ ]:
# From https://dev.socrata.com/foundry/data.cityofnewyork.us/gkne-dk5s
# (select the "Python pandas") tab under "Code Snippets".

import pandas
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofnewyork.us", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.cityofnewyork.us,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# First 20 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("gkne-dk5s", limit=20)

# Convert to pandas DataFrame
dataframe_rest1 = pandas.DataFrame.from_records(results)
print(dataframe_rest1)

A general data request can just use the requests module:

In [ ]:
import pandas
import requests

data_url = 'https://data.cityofnewyork.us/resource/gkne-dk5s.json'

# General data request; include other API keys and credentials as needed in the data argument
response = requests.get(data_url, data={"limit" : "20"})

if response.status_code == 200:
    dataframe_rest2 = pandas.DataFrame.from_records(response.json())    
    print(dataframe_rest2)

## Query an Azure SQL database  <a name="azuresql"></a>

You can access SQL Server databases with the assistance of the pyodbc library.

For example, the article [Use Python to query an Azure SQL database](https://docs.microsoft.com/azure/sql-database/sql-database-connect-query-python) gives you instructions on creating a database containing AdventureWorks data.

The code below, taken from that article, queries the database using pyodbc. To run the code, you must to change the placeholders to identify your specific SQL Server instance and to provide your credentials.

In [ ]:
!pip install pyodbc

In [ ]:
import pyodbc

# Uncomment and modify these four variables for your particular instance
# server = 'your_server.database.windows.net'
# database = 'your_database'
# username = 'your_username'
# password = 'your_password'

server = 'msdocs-notebooks.database.windows.net'
database = 'msdocs-notebooks-adventureworks'
username = 'msdocs-notebooks'
password = 'jupyter12JJ'

driver= '{ODBC Driver 13 for SQL Server}'
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

cursor.execute("SELECT TOP 20 pc.Name as CategoryName, p.name as ProductName FROM [SalesLT].[ProductCategory] pc JOIN [SalesLT].[Product] p ON pc.productcategoryid = p.productcategoryid")
row = cursor.fetchone()

while row:
    print (str(row[0]) + " " + str(row[1]))
    row = cursor.fetchone()

## Use Azure Storage<a name="tablestorage"></a>

Azure Storage provides several different types of non-relational storage, depending on the type of data you have and how you need to access it:

- Table Storage: provides low-cost, high-volume storage for tabular data, such as collected sensor logs, diagnostic logs, and so on.
- Blob storage: provides file-like storage for any type of data.

Azure CosmosDB is also a form of non-relational storage for JSON documents; see the [Other Azure databases](#otherdbs) section for more information on CosmosDB and a variety of other options.

### Prerequisites

- An [Azure storage account](https://azure.microsoft.com/en-us/documentation/articles/storage-create-storage-account/#create-a-storage-account)
- The storage account access key: on the Azure portal, go to the storage account and navigate to **Settings** > **Access keys**. Then copy either **key1** or **key2** and paste into the code in this section.
- Install the azure-storage library (used for tables and blobs)

In [ ]:
!pip install azure-storage

### Table storage

The following code creates a table in the specified Azure Storage account, then adds rows, removes rows, and queries data.

In [ ]:
# Modify these variables with your specific values
azure_storage_account_name = "msdocsnotebooksstorage"
azure_storage_account_key = "KWkfy1CYNRY62XA75QuEjEg7klaAEUmCFLh0Kf+qbGSukMbNEV72HVPq2QYf0KwL9XbipShlP1AeIRBo3j9JdA=="

if azure_storage_account_name is None or azure_storage_account_key is None:
    raise Exception("Provide your specific name and key for your Azure Storage account.")

In [ ]:
from azure.storage.table import TableService
import IPython

# Connect to the table - change the placeholders to your specific names
table_service = TableService(azure_storage_account_name, azure_storage_account_key)

# Create a table
table_name = 'azurenotebookstesttable'
table_service.create_table(table_name)

# Insert entities into the table
entity = {'PartitionKey': 'testItems', 'RowKey': '0', 'age':1}
table_service.insert_entity(table_name, entity)
table_service.insert_entity(table_name, {'PartitionKey': 'testItems', 'RowKey': '10', 'age':2, 'eyecolor':'blue'})

# Query the table
queried_entities = table_service.query_entities(table_name, filter="PartitionKey eq 'testItems'")
print('=== Queried rows after inserts ===')
IPython.display.display_pretty([i for i in queried_entities])

# Delete an entity by using its partition and row key.
table_service.delete_entity(table_name, 'testItems', '0')
                                         
# Query again to show that the entity was removed
queried_entities = table_service.query_entities(table_name, filter="PartitionKey eq 'testItems'")
print('=== Queried rows after delete ===')
IPython.display.display_pretty([i for i in queried_entities])

# Clean up resources
table_service.delete_table('azurenotebookstesttable')

## Access Azure Blobs <a name="blobs"></a>

Blobs store file-like data, which can be private or public. 

The code below demonstrates private keys first. It creates a container, then creates a blob, then reads that blob.

The [shared access](#sharedaccess) section then demonstrates a shared access signature for public read-only access.

You can also put content into blobs using [AzCopy](https://azure.microsoft.com/en-us/documentation/articles/storage-use-azcopy/).

In [ ]:
# Modify these variables with your specific values
azure_storage_account_name = "msdocsnotebooksstorage"
azure_storage_account_key = "KWkfy1CYNRY62XA75QuEjEg7klaAEUmCFLh0Kf+qbGSukMbNEV72HVPq2QYf0KwL9XbipShlP1AeIRBo3j9JdA=="

if azure_storage_account_name is None or azure_storage_account_key is None:
    raise Exception("Provide your specific name and key for your Azure Storage account.")

In [ ]:
from azure.storage.blob import BlockBlobService

# Connect to our blob via the BlobService
blob_service = BlockBlobService(azure_storage_account_name, azure_storage_account_key)

# Create a container
blob_service.create_container('azure-notebooks-data')

# Insider a container, create other containers or blobs
blob_service.create_blob_from_text('azure-notebooks-data', 'sample.txt', 'Hello, Blobs! This is content for the sample.txt file.')

# You can list containers and blobs
containers = blob_service.list_containers()
blobs = blob_service.list_blobs('azure-notebooks-data')

# Read a blob from and get the text; the copy is stored in the Azure Notebooks project
blob_service.get_blob_to_path('azure-notebooks-data', 'sample.txt', 'sample.txt')

# Clean up the created blob and container
blob_service.delete_blob('azure-notebooks-data', 'sample.txt')
blob_service.delete_container('azure-notebooks-data')

Display the contents of the sample blob file:

In [ ]:
!cat sample.txt

## Provide public read-only access to Azure Storage through shared access signatures <a name="sharedaccess"></a>

Sometimes you want to share data from Azure Storage without providing editing capabilities. Shared Access Signatures allow you to share your data and provide whatever level of control you want to the receiver.

The code below creates a shared access signature with read permissions for a table (it also works with blobs). The code then demonstrates the ability to read but not write. Additional permissions are also necessary to query.

In [ ]:
# Modify these variables with your specific values
azure_storage_account_name = "msdocsnotebooksstorage"
azure_storage_account_key = "KWkfy1CYNRY62XA75QuEjEg7klaAEUmCFLh0Kf+qbGSukMbNEV72HVPq2QYf0KwL9XbipShlP1AeIRBo3j9JdA=="

if azure_storage_account_name is None or azure_storage_account_key is None:
    raise Exception("Provide your specific name and key for your Azure Storage account.")

In [ ]:
# Create a container and a blob in that container
from azure.storage.blob import BlockBlobService

# Connect to our blob via the BlobService
blob_service = BlockBlobService(azure_storage_account_name, azure_storage_account_key)

# Create a container
blob_service.create_container('azure-notebooks-data')

# Insider a container, create other containers or blobs
blob_service.create_blob_from_text('azure-notebooks-data', 'sample.txt', 'Hello, Blobs! This is content for the sample.txt file.')


In [ ]:
# Create a shared access signature
from azure.storage.blob.models import BlobPermissions
from datetime import datetime, timedelta

sas_token = blob_service.generate_blob_shared_access_signature(
    'azure-notebooks-data',
    'sample.txt',
    BlobPermissions.READ,
    datetime.utcnow() + timedelta(hours=1)
)

print(sas_token)

In [ ]:
# Create a service and use the shared access signature
sas_blob_service = BlockBlobService(account_name=azure_storage_account_name, sas_token=sas_token, )

sas_blob_service.get_blob_to_text('azure-notebooks-data', 'sample.txt').content

In [ ]:
# Clean up the created blob and container
blob_service.delete_blob('azure-notebooks-data', 'sample.txt')
blob_service.delete_container('azure-notebooks-data')

## Query other Azure databases  <a name="otherdbs"></a>

Azure provides a number of other database types that you can use. The articles below provide guidance for accessing those databases from Python:

- Azure Cosmos DB (fully-indexed NoSQL store for JSON documents):
  - [Build a SQL API app with Python](https://docs.microsoft.com/azure/cosmos-db/create-sql-api-python)
  - [Build a Flask app with the MongoDB API](https://docs.microsoft.com/azure/cosmos-db/create-mongodb-flask)
  - [Create a graph database using Python and the Gremlin API](https://docs.microsoft.com/azure/cosmos-db/create-graph-python)
  - [Build a Cassandra app with Python and Azure Cosmos DB](https://docs.microsoft.com/azure/cosmos-db/create-cassandra-python)
  - [Build a Table API app with Python and Azure Cosmos DB](https://docs.microsoft.com/azure/cosmos-db/create-table-python)
- [Azure Database for PostgreSQL: Use Python to connect and query data](https://docs.microsoft.com/azure/postgresql/connect-python)
- [Quickstart: Use Azure Redis Cache with Python](https://docs.microsoft.com/azure/redis-cache/cache-python-get-started)
- [Azure Database for MySQL: Use Python to connect and query data](https://docs.microsoft.com/azure/mysql/connect-python)
- [Azure Data Factory](https://azure.microsoft.com/en-us/services/data-factory/)
  - [Copy Wizard for Azure Data Factory](https://azure.microsoft.com/en-us/updates/code-free-copy-wizard-for-azure-data-factory/)

Note that for CosmosDB, you can use the [azure-cosmosdb-table](https://pypi.org/project/azure-cosmosdb-table/) library (`!pip install azure-cosmosdb-table`).